In [14]:
import pandas as pd
import nltk
from pyvi import ViTokenizer
import json
import ast

# Extract sentences to train doc2vec

I combine from many data source

- labeled data
- comments crawled from tiki
- features & content of products from tiki

In [19]:
data = []

In [ ]:
# load neg, pos, neu => split to sentences
corpus_paths = ['../data/%s_train.csv' % l for l in ['Positive', 'Neutral', 'Negative']]
for filename in corpus_paths:
    df = pd.read_csv(filename, header=None, names=['text'])
    for _, row in df.iterrows():
        data.extend([s for s in nltk.sent_tokenize(ViTokenizer.tokenize(row['text'])) if len(s) > 1])
print(len(data))

In [ ]:
# load comments & split to sentences
comment_path = '../../crawler-webpage/tiki_crawler/dien-thoai-may-tinh-bang_comments.csv'
comment_df = pd.read_csv(comment_path)
comment_df.head()
for _, row in comment_df.iterrows():
    data.append(ViTokenizer.tokenize(row['title']))
    data.extend([s for s in nltk.sent_tokenize(ViTokenizer.tokenize(row['content'])) if len(s) > 1])
print(len(data))

In [ ]:
# combine features to phrases
category = 'dien-thoai-may-tinh-bang'
file_path = '../../crawler-webpage/tiki_crawler/%s_products.csv' % category
feature_df = pd.read_csv(file_path)
feature_df.head()

In [ ]:
for _, row in feature_df.iterrows():
    data.extend([ViTokenizer.tokenize(f) for f in row['top_features'].split(';')])
    data.extend([s for s in nltk.sent_tokenize(ViTokenizer.tokenize(row['content'])) if len(s) > 1])
    # features
    primary_features = ['%s %s' % (k, v) for k, v in ast.literal_eval(row['features']).items() if k != 'SKU']
    data.extend([ViTokenizer.tokenize(f) for f in primary_features])
print(len(data))

In [ ]:
# export to csv
df = pd.DataFrame.from_dict([{'sentences': s} for s in data])
file_path = '../data/sentences_train.csv'
df.to_csv(file_path, index=False)